In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from lifelines import CoxPHFitter

In [2]:
input_path = "E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed\\"
out_path = input_path + "heatmap\\survival\\"

In [3]:
data = pd.read_csv(input_path + 'fpkm_final_log2-median-centering.csv', sep='\t', index_col=0)
# data = pd.read_csv(input_path + "fpkm_final_log2-zscore_CCL3L1-noDEFB134.csv", sep='\t', header = None)
os = pd.read_csv(input_path + 'Overall_Survival_legacy.txt', sep='\t')
dfs = pd.read_csv(input_path + 'Disease_Free_Survival_legacy.txt', sep='\t')
gender = pd.read_csv(input_path + 'Sex_legacy.txt', sep='\t')

In [4]:
data = data.T
data.reset_index(inplace=True)
data = data.rename(columns={"index":"Sample"})
# data.columns = data.iloc[0]
# data = data.drop(data.index[0])
# data = data.rename(columns={"Gene Name":"Sample"})
# data = data.reset_index(drop=True)

In [5]:
print(data.shape) # (340, 734)
display(data.head(5))
print()
print(os.shape) # (340, 2)
display(os.head(5))
print()
print(dfs.shape) # (340, 2)
display(dfs.head(5))
print()
print(gender.shape) # (340, 2)
display(gender.head(5))
print()

(340, 734)


Gene Name,Sample,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
0,AADA-01A,0.580986,-0.042060,0.013547,-0.734360,0.716873,0.176478,-0.473462,-0.060915,0.120029,...,1.986813,-0.748394,0.791654,0.160138,0.135815,1.214148,-0.239418,0.170838,0.319786,0.819394
1,A5SJ-01A,0.979374,-0.071784,0.098334,0.301992,-0.105847,-0.428254,0.707962,0.268944,0.756201,...,0.911169,0.156321,0.257405,0.027693,0.167687,0.599456,2.378281,0.060643,0.026891,0.039873
2,AACF-01A,-0.436710,-0.055145,-0.007120,-0.439411,-0.342741,0.445181,-0.495030,-3.887819,-0.340829,...,1.964025,-0.724446,0.692565,-0.001063,0.598002,-0.032797,0.049167,-0.642371,-0.133674,0.670390
3,A9GZ-01A,2.521986,0.630880,-0.022410,-0.878293,0.411041,1.067124,-1.441670,0.723679,0.170189,...,2.395214,0.595501,2.513506,-0.013435,-0.185548,-0.227816,-0.164037,0.194186,-0.135175,-0.345550
4,A113-01A,0.771957,-0.012114,-0.000270,-1.283949,-0.609093,-0.882153,-0.216758,-0.961504,-0.117368,...,0.981836,-0.222799,0.806006,0.011926,0.641115,-0.244449,-0.531706,-0.187144,-0.079972,-0.050226



(440, 4)


,Study ID,Patient ID,OS_STATUS,OS_MONTHS
0,lihc_tcga,TCGA-2V-A95S,0:LIVING,NaN
1,lihc_tcga,TCGA-2Y-A9GS,1:DECEASED,23.78
2,lihc_tcga,TCGA-2Y-A9GT,1:DECEASED,53.35
3,lihc_tcga,TCGA-2Y-A9GU,0:LIVING,63.70
4,lihc_tcga,TCGA-2Y-A9GV,1:DECEASED,83.18



(440, 4)


,Study ID,Patient ID,DFS_STATUS,DFS_MONTHS
0,lihc_tcga,TCGA-2V-A95S,0:DiseaseFree,NaN
1,lihc_tcga,TCGA-2Y-A9GS,1:Recurred/Progressed,3.35
2,lihc_tcga,TCGA-2Y-A9GT,1:Recurred/Progressed,35.58
3,lihc_tcga,TCGA-2Y-A9GU,0:DiseaseFree,63.70
4,lihc_tcga,TCGA-2Y-A9GV,1:Recurred/Progressed,71.91



(377, 3)


,Study ID,Patient ID,Sex
0,lihc_tcga,TCGA-2V-A95S,Male
1,lihc_tcga,TCGA-2Y-A9GS,Male
2,lihc_tcga,TCGA-2Y-A9GT,Male
3,lihc_tcga,TCGA-2Y-A9GU,Female
4,lihc_tcga,TCGA-2Y-A9GV,Female


### Subset data to keep only genes from Sangro's list

In [6]:
lsangro = pd.read_csv(input_path + 'Sangro_Table2_GeneList.csv', sep='\t', header=None)
lsangro.rename(columns={0:"Gene Name"}, inplace=True)
print(lsangro.shape)
display(lsangro.head(5))

(29, 1)


,Gene Name
0,CD274(PDL1)
1,CD8A
2,LAG3
3,STAT1
4,CCL2


In [7]:
lsangro.iloc[0,0] = "CD274" # 'CD274(PDL1)'
lsangro.drop(lsangro[lsangro["Gene Name"]=="CCL3"].index, inplace=True)
print(lsangro.shape)
display(lsangro.head(5))

(28, 1)


,Gene Name
0,CD274
1,CD8A
2,LAG3
3,STAT1
4,CCL2


In [8]:
lsangro = lsangro["Gene Name"].values.tolist()
lsangro.insert(0, "Sample")
data = data[lsangro]
print(data.shape) # (340, 29)
display(data.head(5))

(340, 29)


Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,CCL5,CD27,CXCR6,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1
0,AADA-01A,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,...,1.904710,1.187266,0.555145,-0.272298,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193
1,A5SJ-01A,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,...,1.409919,1.645123,0.422194,-0.419198,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949
2,AACF-01A,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,...,0.010616,-0.480303,0.036483,-0.062031,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411
3,A9GZ-01A,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,...,-1.530659,0.025447,-0.204613,-0.273227,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765
4,A113-01A,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,...,-0.292722,0.459020,0.007975,-0.471770,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989


### Subset genes to keep only genes used in the Sum Up version

In [9]:
# df_gene_sum = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\heatmap\\gene_clusters_sum_filter-4.csv', sep=',')

In [10]:
# print(df_gene_sum.shape) # (340, 2)
# display(df_gene_sum.head(5))
# print()

In [11]:
# lgene_sum = df_gene_sum[(df_gene_sum["Cluster"]=="Cluster 02") | (df_gene_sum["Cluster"]=="Cluster 01")]["Gene"].values.tolist()
# print(lgene_sum)

In [12]:
# lgene_sum.insert(0, "Sample")
# data = data[lgene_sum]
# print(data.shape)
# display(data.head(5)) # (340, 54)

### Extract survival data

In [13]:
sample = data["Sample"][0]

In [14]:
# Here is no chained operation. SettingWithCopyWarning maybe be caused by created item column, try to build the df again.
def exact_info(df, item):
    i=0
    data[item] = ''
    for sample in data["Sample"]:
        info = set(df[df['Patient ID'].str.contains(sample[:-4][-4:])][item])
        if len(info) != 1 :
#             raise ValueError('Multiple or no values found.')
            i=i+1
        else:
            info = list(info)[0]
            data.loc[data["Sample"]==sample, item] = info
    return i # count of samples not found

In [15]:
print(exact_info(df = os, item = 'OS_STATUS')) # 0
print(exact_info(df = os, item = 'OS_MONTHS')) # 0

0
0


In [16]:
print(exact_info(df = dfs, item = 'DFS_STATUS')) # 0
print(exact_info(df = dfs, item = 'DFS_MONTHS')) # 0

0
0


In [17]:
print(exact_info(df = gender, item = 'Sex')) # 0

0


In [18]:
print(data.shape) # (340, 33)
display(data.head(5))
print()

(340, 34)


Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS,Sex
0,AADA-01A,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,...,0.258454,0.796701,-0.189377,1.844013,0.856193,0:LIVING,40.51,0:DiseaseFree,40.51,Female
1,A5SJ-01A,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,...,0.628489,0.699625,1.558460,0.630920,0.088949,0:LIVING,22.93,1:Recurred/Progressed,10.12,Male
2,AACF-01A,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,...,-0.040435,-0.068163,1.191091,0.072777,0.474411,1:DECEASED,11.99,1:Recurred/Progressed,3.91,Male
3,A9GZ-01A,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,...,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1:DECEASED,27.86,1:Recurred/Progressed,23.62,Female
4,A113-01A,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,...,0.159172,0.014350,-0.002096,-0.582563,-0.580989,0:LIVING,79.66,1:Recurred/Progressed,55.06,Female


### Preprocess

In [19]:
for i in range(len(data)):
    data.loc[i,'OS_STATUS'] = data.loc[i,'OS_STATUS'][0]
    try:
        data.loc[i,'DFS_STATUS'] = data.loc[i,'DFS_STATUS'][0]
    except:
        pass

In [20]:
display(data.dtypes[-4:])
data = data.astype({'OS_STATUS':'float64', 'OS_MONTHS':'float64', 'DFS_STATUS':'float64', 'DFS_MONTHS':'float64'})
display(data.dtypes[-4:])

Gene Name
OS_MONTHS     object
DFS_STATUS    object
DFS_MONTHS    object
Sex           object
dtype: object

Gene Name
OS_MONTHS     float64
DFS_STATUS    float64
DFS_MONTHS    float64
Sex            object
dtype: object

In [21]:
print(data.shape) # (340, 33)
display(data.head(5))
print()

(340, 34)


Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS,Sex
0,AADA-01A,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,...,0.258454,0.796701,-0.189377,1.844013,0.856193,0.0,40.51,0.0,40.51,Female
1,A5SJ-01A,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,...,0.628489,0.699625,1.558460,0.630920,0.088949,0.0,22.93,1.0,10.12,Male
2,AACF-01A,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,...,-0.040435,-0.068163,1.191091,0.072777,0.474411,1.0,11.99,1.0,3.91,Male
3,A9GZ-01A,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,...,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1.0,27.86,1.0,23.62,Female
4,A113-01A,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,...,0.159172,0.014350,-0.002096,-0.582563,-0.580989,0.0,79.66,1.0,55.06,Female


In [22]:
# # Export data
# data.to_csv(input_path+"data_median_survival.csv",sep="\t", index=False)

### Explore data

In [23]:
# Only 1 missing data for OS, with OS_STATUS but no OS_MONTHS
data[data['OS_MONTHS'].isna()]

Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS,Sex
33,A95S-01A,0.46826,3.238845,2.840833,4.188422,0.414246,1.822712,5.929623,4.996384,2.401558,...,1.623585,1.848018,4.139949,3.163895,2.876739,0.0,NaN,0.0,NaN,Male


In [24]:
# Censored values
data.groupby(["OS_STATUS"]).count()

Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_MONTHS,DFS_STATUS,DFS_MONTHS,Sex
OS_STATUS,,,,,,,,,,,,,,,,,,,,,
0.0,220,220,220,220,220,220,220,220,220,220,...,220,220,220,220,220,220,219,220,219,220
1.0,120,120,120,120,120,120,120,120,120,120,...,120,120,120,120,120,120,120,77,77,120


In [25]:
# Only 1 missing data for OS, with OS_STATUS but no OS_MONTHS
data[data['Sex'].isna()]

Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS,Sex


In [26]:
# Censored values
data.groupby(["Sex"]).count()

Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
Sex,,,,,,,,,,,,,,,,,,,,,
Female,110,110,110,110,110,110,110,110,110,110,...,110,110,110,110,110,110,110,110,92,92
Male,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,229,205,204


## Overall survival

### Visualize censored values

In [27]:
# ax = plot_lifetimes(T, event_observed=E)
# # ax.set_xlim(0, 25)
# # ax.vlines(10, 0, 30, lw=2, linestyles='--')
# ax.set_xlabel("time")
# ax.set_title("Overall survival time")
# # print("Observed lifetimes at time %d:\n" % (CURRENT_TIME), observed_lifetimes)

### Cox proportional hazards regression model

#### Cox with only Sangro's list

In [28]:
data_dummy = data.dropna(subset = ["OS_MONTHS"]).reset_index(drop=True).drop(columns=["Sample", "DFS_STATUS", "DFS_MONTHS"])
print(data_dummy.shape) # (339, 30)
data_dummy.head(5)

(339, 31)


Gene Name,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,HLA-DMA,...,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,Sex
0,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,-0.455090,...,-0.272298,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193,0.0,40.51,Female
1,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,0.351409,...,-0.419198,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949,0.0,22.93,Male
2,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,0.000915,...,-0.062031,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411,1.0,11.99,Male
3,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,-1.604050,...,-0.273227,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1.0,27.86,Female
4,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,-0.984838,...,-0.471770,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989,0.0,79.66,Female


In [29]:
# Using Cox Proportional Hazards model without on hot encoding
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy, "OS_MONTHS", "OS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.print_summary()    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


In [30]:
cph.summary[cph.summary["p"]<0.05]

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CCL4,0.433703,1.542961,0.216924,0.00854,0.858867,1.008576,2.360484,1.999332,0.045572,4.455694


In [31]:
# Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD8A","LAG3","STAT1","OS_MONTHS", "OS_STATUS", "Sex"]], "OS_MONTHS", "OS_STATUS", 
              strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.451818,0.63647,0.189457,-0.823147,-0.08049,0.439048,0.922664,-2.384813,0.017088,5.87089


In [32]:
# Gajewski 13-Gene Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL2","CCL4","CD8A","CXCL10","CXCL9","GZMK","HLA-DMA","HLA-DMB","HLA-DOA","HLA-DOB","ICOS","IRF1",
                          "OS_MONTHS", "OS_STATUS", "Sex"]], "OS_MONTHS", "OS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CCL4,0.439272,1.551577,0.194690,0.057686,0.820857,1.059383,2.272447,2.256263,0.024054,5.377568
CD8A,-0.529562,0.588863,0.248177,-1.015979,-0.043144,0.362048,0.957774,-2.133808,0.032859,4.927588
GZMK,-0.363048,0.695553,0.216375,-0.787135,0.061040,0.455147,1.062941,-1.677863,0.093374,3.420839
ICOS,1.551358,4.717873,0.603349,0.368816,2.733900,1.446022,15.392799,2.571246,0.010133,6.624750


In [33]:
# 6-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CXCL10","CXCL9","HLA-DRA","IDO1","IFNG","STAT1","OS_MONTHS", "OS_STATUS", "Sex"]], "OS_MONTHS", 
              "OS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CXCL9,-0.186363,0.829972,0.113283,-0.408394,0.035667,0.664717,1.036310,-1.645117,0.099946,3.322712
HLA-DRA,-0.162970,0.849616,0.088609,-0.336640,0.010699,0.714166,1.010757,-1.839216,0.065883,3.923942
IDO1,0.242217,1.274070,0.135775,-0.023898,0.508331,0.976385,1.662514,1.783951,0.074432,3.747942


In [34]:
# Interferon Gamma Biology signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL5","CD27","CXCL9","CXCR6","IDO1","STAT1","OS_MONTHS", "OS_STATUS", "Sex"]], "OS_MONTHS", 
              "OS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
IDO1,0.294819,1.342883,0.137918,0.024504,0.565133,1.024806,1.759683,2.137632,0.032547,4.941347


In [35]:
# T-cell Exhaustion signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD276","CD8A","LAG3","PDCD1LG2","TIGIT","OS_MONTHS", "OS_STATUS", "Sex"]], "OS_MONTHS", 
              "OS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD276,-0.243126,0.784173,0.130018,-0.497956,0.011704,0.607772,1.011773,-1.869943,0.061492,4.023464
CD8A,-0.858946,0.423608,0.264584,-1.377521,-0.340371,0.252203,0.711506,-3.246401,0.001169,9.740828
TIGIT,1.163825,3.202159,0.658022,-0.125875,2.453525,0.881725,11.629271,1.768671,0.076949,3.699958


In [36]:
# Ribas 10-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCR5","CXCL10","CXCL11","CXCL9","GZMA","HLA-DRA","IDO1","IFNG","PRF1","STAT1","OS_MONTHS", 
                          "OS_STATUS", "Sex"]], "OS_MONTHS", "OS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
IDO1,0.307681,1.360267,0.143644,0.026144,0.589218,1.026489,1.802577,2.141968,0.032196,4.956972
IFNG,0.758025,2.134057,0.383331,0.006709,1.509341,1.006732,4.523747,1.977467,0.047989,4.381155


In [37]:
# cph.summary[cph.summary["coef"]>2]

In [38]:
# # Check if the proprtional hazards assuption is violated
# cph.check_assumptions(data_dummy, show_plots=True)

## Disease free survival

### Cox proportional hazards regression model

#### Cox with only Sangro's list

In [39]:
data_dummy = data.dropna(subset = ["DFS_MONTHS"]).reset_index(drop=True).drop(columns=["Sample", "OS_STATUS", "OS_MONTHS"])
print(data_dummy.shape) # (296, 30)
data_dummy.head(5)

(296, 31)


Gene Name,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,HLA-DMA,...,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,DFS_STATUS,DFS_MONTHS,Sex
0,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,-0.455090,...,-0.272298,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193,0.0,40.51,Female
1,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,0.351409,...,-0.419198,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949,1.0,10.12,Male
2,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,0.000915,...,-0.062031,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411,1.0,3.91,Male
3,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,-1.604050,...,-0.273227,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1.0,23.62,Female
4,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,-0.984838,...,-0.471770,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989,1.0,55.06,Female


In [40]:
# Using Cox Proportional Hazards model without on hot encoding
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy, "DFS_MONTHS", "DFS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.print_summary()    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


In [41]:
cph.summary[cph.summary["p"]<0.05]

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CCL4,0.368025,1.444878,0.183716,0.007949,0.728101,1.007981,2.071144,2.003232,0.045152,4.469053
HLA-DRA,-0.634151,0.530386,0.205807,-1.037525,-0.230777,0.354331,0.793917,-3.081290,0.002061,8.922401
IDO1,0.293081,1.340551,0.147067,0.004835,0.581327,1.004846,1.788410,1.992838,0.046279,4.433493
PRF1,-0.898377,0.407230,0.307027,-1.500138,-0.296616,0.223099,0.743330,-2.926055,0.003433,8.186356


In [42]:
# Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD8A","LAG3","STAT1","DFS_MONTHS", "DFS_STATUS", "Sex"]], "DFS_MONTHS", "DFS_STATUS", 
              strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.460493,0.630972,0.169748,-0.793194,-0.127793,0.452398,0.880036,-2.712802,0.006672,7.227730
LAG3,0.289769,1.336119,0.153000,-0.010105,0.589643,0.989946,1.803345,1.893917,0.058236,4.101944


In [43]:
# Gajewski 13-Gene Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL2","CCL4","CD8A","CXCL10","CXCL9","GZMK","HLA-DMA","HLA-DMB","HLA-DOA","HLA-DOB","ICOS","IRF1",
                          "DFS_MONTHS", "DFS_STATUS", "Sex"]], "DFS_MONTHS", "DFS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.635286,0.529784,0.206324,-1.039674,-0.230899,0.353570,0.793820,-3.07907,0.002076,8.911647
ICOS,1.347309,3.847058,0.525629,0.317094,2.377523,1.373132,10.778177,2.56323,0.010370,6.591393


In [44]:
# 6-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CXCL10","CXCL9","HLA-DRA","IDO1","IFNG","STAT1","DFS_MONTHS", "DFS_STATUS", "Sex"]], "DFS_MONTHS", 
              "DFS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
HLA-DRA,-0.293340,0.745769,0.076318,-0.442920,-0.143760,0.642159,0.866096,-3.843670,0.000121,13.010229
IDO1,0.318914,1.375633,0.129959,0.064199,0.573629,1.066304,1.774696,2.453957,0.014129,6.145157


In [45]:
# Interferon Gamma Biology signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL5","CD27","CXCL9","CXCR6","IDO1","STAT1","DFS_MONTHS", "DFS_STATUS", "Sex"]], "DFS_MONTHS", 
              "DFS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
IDO1,0.228756,1.257035,0.132329,-0.030604,0.488115,0.96986,1.629243,1.728692,0.083864,3.5758


In [46]:
# T-cell Exhaustion signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD276","CD8A","LAG3","PDCD1LG2","TIGIT","DFS_MONTHS", "DFS_STATUS", "Sex"]], "DFS_MONTHS", 
              "DFS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.718708,0.487381,0.214062,-1.138263,-0.299154,0.320375,0.741445,-3.357473,0.000787,10.312112
TIGIT,1.018118,2.767981,0.572236,-0.103445,2.139681,0.901726,8.496725,1.779191,0.075208,3.732962


In [47]:
# Ribas 10-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCR5","CXCL10","CXCL11","CXCL9","GZMA","HLA-DRA","IDO1","IFNG","PRF1","STAT1","DFS_MONTHS", 
                          "DFS_STATUS", "Sex"]], "DFS_MONTHS", "DFS_STATUS", strata="Sex"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
GZMA,0.424184,1.528342,0.182679,0.066140,0.782227,1.068377,2.186336,2.322022,0.020232,5.627235
HLA-DRA,-0.274898,0.759650,0.099772,-0.470447,-0.079349,0.624723,0.923718,-2.755264,0.005864,7.413781
IDO1,0.405346,1.499821,0.138812,0.133278,0.677413,1.142568,1.968778,2.920097,0.003499,8.158749
PRF1,-0.848039,0.428254,0.243139,-1.324583,-0.371494,0.265914,0.689703,-3.487870,0.000487,11.004133


In [48]:
# # Check if the proprtional hazards assuption is violated
# cph.check_assumptions(data_dummy, show_plots=True)

In [49]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [50]:
# Compute and view VIF
vif = pd.DataFrame()
vif["variables"] = data_dummy.iloc[:,:-2].columns
vif["VIF"] = [variance_inflation_factor(data_dummy.iloc[:,:-2].values, i) for i in range(data_dummy.iloc[:,:-2].shape[1])]

# View results using print
print(vif)

     variables        VIF
0        CD274   1.978433
1         CD8A  16.528580
2         LAG3   7.742435
3        STAT1   2.328270
4         CCL2   2.361129
5         CCL4   5.841564
6       CXCL10   4.406277
7        CXCL9   5.349496
8         GZMK  11.868901
9      HLA-DMA   8.945539
10     HLA-DMB  22.172437
11     HLA-DOA   8.497410
12     HLA-DOB   3.621173
13        ICOS  14.330293
14        IRF1   3.425412
15     HLA-DRA  14.607995
16        IDO1   2.341383
17        IFNG   6.754137
18        CCL5  17.343016
19        CD27  18.392496
20       CXCR6  11.633019
21       CD276   1.531900
22    PDCD1LG2   4.334633
23       TIGIT  25.940188
24        CCR5  12.296135
25      CXCL11   3.553870
26        GZMA  22.428491
27        PRF1  10.834269
28  DFS_STATUS   1.437127


In [51]:
str(vif["variables"].values.tolist()).replace("', '","+").replace("-",".")

"['CD274+CD8A+LAG3+STAT1+CCL2+CCL4+CXCL10+CXCL9+GZMK+HLA.DMA+HLA.DMB+HLA.DOA+HLA.DOB+ICOS+IRF1+HLA.DRA+IDO1+IFNG+CCL5+CD27+CXCR6+CD276+PDCD1LG2+TIGIT+CCR5+CXCL11+GZMA+PRF1+DFS_STATUS']"